In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms, utils, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from torch.autograd import Variable
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import torch
import numpy
import re
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataloader import default_collate
from transformers import XLMRobertaTokenizer, XLMRobertaXLModel
from transformers import XLNetTokenizer, XLNetModel
import random
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision.ops.focal_loss as fl
import tensorflow_addons as tfa
import gc

In [2]:
TRAIN1_DIR = '/kaggle/input/foody-data-after-pre/pre_train.csv'
TRAIN2_DIR = '/kaggle/input/foody-data-after-pre/data2_processed.csv'
TEST_DIR = '/kaggle/input/foody-data-after-pre/pre_test.csv'
VAL_DIR = '/kaggle/input/foody-data-after-pre/val_labelled.csv'

In [3]:
torch.manual_seed(25)
np.random.seed(25)
random.seed(25)

cudnn.benchmark = True
cudnn.deterministic = True

In [4]:
df = pd.read_csv(TRAIN1_DIR)

In [5]:
df = df.drop("Unnamed: 0", axis = 1)

In [6]:
df['Rating'] = df['Rating'].astype(int)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
df['Comment'] = df['Comment'].str.strip()

In [9]:
df = df[df['Comment']  != 'None']

In [10]:
# from sklearn.model_selection import train_test_split
# df_train, df_test = train_test_split(df, test_size=0.15, random_state=42)

df_train = df
df_val = pd.read_csv(VAL_DIR)

In [11]:
df_val['Rating'] = df_val['Rating'].astype(int)

In [12]:
df_2 = pd.read_csv(TRAIN2_DIR)
df_2 = df_2.drop("Unnamed: 0", axis = 1)

In [13]:
df_2['Rating'] = df_2['Rating'].astype(int)

In [14]:
df_0_2 = df_2[df_2['Rating'] == 0].sample(1450)

In [15]:
df_0_2['Rating'].value_counts()

0    1450
Name: Rating, dtype: int64

In [16]:
df_1_2 = df_2[df_2['Rating'] == 1].sample(1000)

In [17]:
df_train = pd.concat([df_train[['Comment','Rating']],
                df_0_2[['Comment','Rating']],
                df_1_2[['Comment','Rating']]
               ])

In [18]:
df_train.shape

(11519, 2)

In [19]:
df_val['Rating'] = df_val['Rating'].astype(int)

In [20]:
from sklearn.utils import shuffle

In [21]:
df_train = df_train.drop_duplicates(['Comment'])

In [22]:
df_train['Rating'].value_counts()

1    8122
0    3348
Name: Rating, dtype: int64

### model

In [23]:
from tqdm import tqdm

# wonrax/phobert-base-vietnamese-sentiment
class BERT_classification(nn.Module):
    def __init__(self):
        super(BERT_classification, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
        self.classifier = nn.Sequential(
          nn.Dropout(p=0.2),
          nn.Linear(in_features=768, out_features = 256, bias=True).to(device),
          nn.Dropout(p=0.1),
          nn.Linear(in_features=256, out_features = 2, bias=True).to(device)
        )
        self.criterior = nn.CrossEntropyLoss()
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        param_optimizer = list(self.bert.named_parameters())
        self.optimizer =optim.AdamW(
            [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            {'params': self.classifier.parameters(), 'lr': 8e-5}],
            lr = 2e-5
        )

    def tokenize(self,v_text, label):
        v_tokenized = []
        max_len = 256
        for i_text in v_text:
            i_text = standardize_data(i_text)
            i_text = self.choose_sub_text(i_text)
            line = self.tokenizer.encode(i_text, max_length = max_len, truncation=True)
            v_tokenized.append(line)

        padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
        attention_mask = numpy.where(padded == 1, 0, 1)
        padded = torch.tensor(padded).to(torch.long)
        attention_mask = torch.tensor(attention_mask)
        
        target = torch.tensor(label) 
        data_tensor = TensorDataset(padded, target) 
        train_loader = DataLoader(dataset = data_tensor, batch_size = 32)
        attention_mask = DataLoader(attention_mask, batch_size = 32)
        
        return train_loader, attention_mask
    
    def choose_sub_text(self, text):
        tmp = len(text.split())
        if tmp > 256:
            words = text.split()
            text = ' '.join(words[0:210]) + ' ' + ' '.join(words[-46:])
            
        return text
    
    def forward(self, input_ids, attention_mask):
        _, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
             return_dict=False
        )
        output = self.classifier(output)
        return output

In [24]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace("  "," ").replace("  "," ") \
        .replace(')','').replace('(','') \
        .replace("—",' ').replace("\n",' '). replace("\t",' ') \
        .replace('   ', ' ').replace('  ', ' ')
            
    return row

In [25]:
def train_model_bert(bert, save_path, df_train, df_test, num_epochs=1, type_model = 'bert', check_fold = 1):
        text_train = df_train['Comment'].to_list()
        label_train = df_train['Rating'].to_list()
        text_test = df_test['Comment'].to_list()
        label_test = df_test['Rating'].to_list()
        
        Data_train, attention_train = bert.tokenize(text_train, label_train)
        Data_test, attention_test = bert.tokenize(text_test, label_test)
        
        dataloader_dict = {
            'train' : Data_train,
            'test' : Data_test
        }
        
        check = 0
        
        for epoch in range(num_epochs + 1):
            if epoch == 0:
                print()
                continue
            print("Epoch {}/{} ".format(epoch, num_epochs))

            for phase in ['train', 'test']:
                if phase == 'train':
                    bert.train()
                    attention = attention_train
                else:
                    bert.eval()
                    attention = attention_test

                epoch_loss = 0.0
                epoch_corrects = 0

                for (inputs, labels), i_attention in tqdm(zip(dataloader_dict[phase], attention)):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    i_attention = i_attention.to(device)

                    bert.optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = bert(inputs,i_attention)

                        loss = bert.criterior(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            bert.optimizer.step()

                        epoch_loss += loss.item() * inputs.size(0)
                        epoch_corrects += torch.sum(preds == labels.data)
                        del outputs
                        torch.cuda.empty_cache()

                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                
                print("{} Loss: {:.4f}  Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
                
                if check < epoch_acc and phase == 'test':
                    check = epoch_acc
                    torch.save(bert.state_dict(), "./full_{}_model_fold{}.pth".format(type_model, check_fold))

### train

In [26]:
df_train = shuffle(df_train)

In [27]:
b = BERT_classification()
train_model_bert(b, './', df_train, df_val, num_epochs = 6, type_model = 'bert')

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

test Loss: 0.2483  Acc: 0.9306
Epoch 6/6 


359it [06:07,  1.02s/it]


train Loss: 0.1266  Acc: 0.9611


160it [00:40,  3.99it/s]

test Loss: 0.2634  Acc: 0.9275


In [28]:
def predict_bert(model, df_val):
    text_sub = df_val['Comment'].to_list()
    tmp = np.zeros(len(text_sub))
    net = model
    net.eval()
    Data_sub, attention_sub = net.tokenize(text_sub, tmp)
    attention = attention_sub

    list_tmp = []
    list_truth = np.array([])

    for (inputs, labels), i_attention in tqdm(zip(Data_sub, attention)):
        inputs = inputs.to(device)
        labels = labels.to(device)
        i_attention = i_attention.to(device)
        with torch.no_grad():

            outputs = net(inputs,i_attention)

            list_tmp.append(outputs)

            _ , preds = torch.max(outputs, 1)

            del outputs
            torch.cuda.empty_cache()
            gc.collect()

            list_truth = np.append(list_truth,preds.cpu().numpy())
    
    list_prob = []
    for i in range(len(list_tmp)):
        probs = F.softmax(list_tmp[i], dim=1) # assuming logits has the shape [batch_size, nb_classes]
        list_prob.append(probs)
    
    list_ans = []
    for i in list_prob:
        list_ans = np.append(list_ans,i.cpu().numpy()[:,1])
        
    return list_truth, list_ans

In [29]:
b = BERT_classification()
w = torch.load('/kaggle/working/full_bert_model_fold{}.pth'.format(1))
b.load_state_dict(w)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [30]:
import torch.nn.functional as F
# text_test = df_val['Comment'].to_list()
list_predict_bert, list_ans = predict_bert(b, df_val)

160it [01:15,  2.13it/s]


### submit

In [32]:
df_sub = pd.read_csv(TEST_DIR)

In [33]:
import torch.nn.functional as F

b = BERT_classification()
w = torch.load('/kaggle/working/full_bert_model_fold{}.pth'.format(1))
b.load_state_dict(w)
list_predict, list_ans1 = predict_bert(b, df_sub)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
160it [01:14,  2.13it/s]


In [34]:
list_ans1

array([0.29014805, 0.05337064, 0.97625148, ..., 0.0036939 , 0.91400576,
       0.98500466])

In [38]:
df_sub['Rating'] = list_ans1

In [39]:
df_sub[['RevId','Rating']].to_csv('submission_phobert.csv',index = False)